In [2]:
from random import randint, getrandbits
import itertools
import datetime
import tensorflow.experimental.numpy as tnp
import timeit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from Final_ShreyaNNtraining_Final_DONT_TOUCH import random_mutation, random_n_mutation, random_delete, random_insert, sample_from_transcripts
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU, Bidirectional, Activation, MaxPool1D, Conv1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

%load_ext tensorboard

In [3]:
tf.__version__

'2.5.0'

In [ ]:
'''
def shuffle_x_y_together(a,b):
    a_s = a.shape[0]
    b_s = b.shape[0]
    assert a_s == b_s, "Arrays must be of same size"
    i = np.argsort(np.random.permutation(a_s))
    return (a[i,:,:],b[i,:])

def build_train_sets(data_samplers,
                  sample_size=1000,
                  max_fragment_length=400,
                    test_size = .2):

    test_x = None
    train_x = None
    X_train = None
    y_train = []
    X_test = None
    y_test = []


    #TRAIN SET
    for sampler in data_samplers:
        tmp_x = sampler.build_tensor(sample_size)
        if X_train is not None:
            X_train = np.concatenate((tmp_x, X_train))
        else:
            X_train = tmp_x
            
        y_train = y_train + [sampler.training_output for i in range(sample_size)]
        
    y_train = np.asarray(y_train)
    X_train,y_train = shuffle_x_y_together(X_train,y_train)
        
    #TEST SET
    for sampler in data_samplers:
        tmp_x = sampler.build_tensor(int(sample_size * test_size))
        if X_test is not None:
            X_test = np.concatenate((tmp_x, X_test))
        else:
            X_test = tmp_x
            
        y_test = y_test + [sampler.training_output for i in range(int(sample_size * test_size))]
        
    y_test = np.asarray(y_train)
    X_test,y_test = shuffle_x_y_together(X_test,y_test)
    
    return X_train, y_train, X_test, y_test
    
'''

In [ ]:
'''
def print_log(file_path,epoch):
    f = open(file_path,'a')
    f.write('Trained epoch: '+ epoch +'\n') # python will convert \n to os.linesep
    f.close()

# Log Training
def train_log(filename, epoch, accuracy):
    f = open(file_path,'a')
    f.write('Trained file: '+str(filename)+', epoch '+str(epoch)+ ' accuracy:'+ str(accuracy)+'\n') # python will convert \n to os.linesep
    f.close()
'''

In [4]:
output_classes = {
    'hla_a'  :1,
    #'hla_b'  :1,
    #'hla_c'  :2,
    'hla_dp' :2,
    'hla_dq' :3,
    'hla_dr' :4,
    #'neither': 5,
    #'hla_dr_copy': 5,
    #'hla_tap':6,
    #'hla_mica':7,
    #'hla_micb':8,
    #'hla_kir':9
    #'refMrna' :10
    #'hla': 1,
    'non-hla': 0
}

#3-way:
#hla_dp: 2
#hla_dq: 3
#non_hla: 0

#4-way:
#hla_a: 1
#hla_dp: 2
#hla_dq: 3
#non_hla: 0

#5-way:
#hla_a: 1
#hla_dp: 2
#hla_dq: 3
#hla_dr: 4
#non_hla: 0



output_paths= {
    'hla_a'  :"data/Sample/hla_a2.fastm",
    #'hla_b'  :"/Users/davidott/Workspace/notebooks/test/hla_fastm/hla_b.fastm",
    #'hla_c'  :"/Users/davidott/Workspace/notebooks/test/hla_fastm/hla_c.fastm",
    'hla_dp' :"data/Sample/hla_dp2.fastm",
    'hla_dq' :"data/Sample/hla_dq2.fastm",
    'hla_dr' :"data/Sample/hla_dr2.fastm",
    #'hla_tap':"/Users/davidott/Workspace/notebooks/test/hla_fastm/hla_tap.fastm",
    #'hla_mica':"/Users/davidott/Workspace/notebooks/test/hla_fastm/hla_mica.fastm",
    #'hla_micb':"/Users/davidott/Workspace/notebooks/test/hla_fastm/hla_micb.fastm",
    #'hla_kir':"/root/Notebooks/test/hla_fastm/KIR_nuc.fastm"
    'non-hla': "data/Sample/refMrna.fastm"
    
}

file_paths= {
    #'hla_a'  :np.asarray([1,0,0,0,0,0,0,0,0]),
    #'hla_b'  :np.asarray([0,1,0,0,0,0,0,0,0]),
    #'hla_c'  :np.asarray([0,0,1,0,0,0,0,0,0]),
    #'hla_dp' :np.asarray([0,0,0,1,0,0,0,0,0]),
    #'hla_dq' :np.asarray([0,0,0,0,1,0,0,0,0]),
    #'hla_dr' :np.asarray([0,0,0,0,0,1,0,0,0]),
    #'hla_tap':np.asarray([0,0,0,0,0,0,1,0,0]),
    #'hla_mica':np.asarray([0,0,0,0,0,0,0,1,0]),
    #'hla_micb':np.asarray([0,0,0,0,0,0,0,0,1]),
    #'hla_kir':np.asarray([0,0,0,0,0,0,0,0,1])
    'hla_a':np.asarray([1,0,0,0,0]),
    'hla_dp':np.asarray([0,1,0,0,0]),
    'hla_dq':np.asarray([0,0,1,0,0]),
    'hla_dr':([0,0,0,1,0]),
    #'hla_dr_copy':np.asarray([0,0,0,0,1])
    'non-hla': ([0,0,0,0,1])
    #'neither': ([0,0,0,0,0,0])
}

mp = {
    
        'layers':
        [
            'conv_1',
            'maxpool_1',
            'dropout_1',
            'bidirectional_1',
            'dropout_2',
            'dense_1',
            'dense_drop_1',
            'dense_2',
            'dense_drop_2',
            'dense_3'
        ],
        
        'conv_1': 
        { 
            'type':'Conv1D',
            'num_filters': 320,
            'kernel_size': 9,
            'stride': 3,
            'name': '1dconv_1',
            'activation':'relu', 
        },
        'maxpool_1': 
        {
            'type':'MaxPooling1D',
            'pool_size': 3,
            'strides': None,
            'name': 'maxpool_1'
        },
        'dropout_1': 
        {
            'type':'Dropout',
            'percent': .4,
            'name': 'dropout_1',
        },
        'bidirectional_1':
        {
            'type':'Bidirectional',
            'output_dim': 320,
            'recurrent_dropout':.3,
            'dropout':.3,
            'name':'bidirectional_lstm1',
            'return_sequences':False,
            'merge_mode':'concat'
        },
        'dropout_2': 
        {
            'type':'Dropout',
            'percent': .6,
            'name': 'dropout_2',
        },
        'dense_1':
        {
            'type':'Dense',
            'activation':'relu',
            'output_dim': 160,
            'name': 'multiclass_1'
        },
        'dense_drop_1':
        {
            'type':'Dropout',
            'percent': .4,
            'name': 'dense_dropout_1',
        },
        'dense_2':
        {
            'type':'Dense',
            'activation':'relu',
            'output_dim': 160,
            'name': 'multiclass_dense_2'
        },
        'dense_drop_2':
        {
            'type':'Dropout',
            'percent': .4,
            'name': 'dense_dropout_2',
        },
        'dense_3':
        {
            'type':'Dense',
            'activation':'sigmoid',
            'output_dim': len(output_classes.keys()),
            'name': 'multiclass_dense_3'
        },
        'max_fragment_length':400,
        'loss':'binary_crossentropy',
        'optimizer':'rmsprop',
        'metrics':['accuracy'],
        'num_classes': len(output_classes.keys())
    }
    
training_params = {
    'num_samples': 1000,
    'batch_size': 100,
    'nb_epochs' : 2000,
    'learning_rate': 1e-4,
    'n_prob':.06,
    'mutation_prob':.02,
    'delete_prob':.015,
    'insert_prob':.015,
    'min_read_length':75,
    'max_read_length': 128,
    'b_randomize_location':True,
    'b_randomize_direction':False, 
    'sample_size': 10
}

output_length= len(output_paths)

## Data Processing

In [5]:
#Converting data into numpy array
def reads_to_numpy(input):#reads, training_parameters):
    final_array = []
    i = 0
    for index in input:
        string_array = []
        #print(index)
        #[final_array.append(string_array) for index in input]
        for i in index:
            #print(i)
            if i == "A":
                string_array.append([1,0,0,0])
            elif i == "C":
                string_array.append([0,1,0,0])
            elif i == "G":
                string_array.append([0,0,1,0])
            elif i == "T":
                string_array.append([0,0,0,1])
            else:
                string_array.append([0,0,0,0])
        final_array.append(string_array)
        tensor_array= np.array(final_array)
    return tensor_array    

#tensor_array = reads_to_numpy(transcript_array)
#np.save('/Users/shreyaramesh/Downloads/Shreya_Data/hla_fastm_copy/final_numpyarray', tensor_array)
#print(len(transcript_array))
#print(tensor_array)
#print(tensor_array.shape)

In [5]:
#Importing hla a,dp,dq,dr and non-hla data and appending both into one file
total_tensor_class = []
total_tensor_array = []

for index in output_paths:
    temp_array = []
    temp_combo = sample_from_transcripts(output_paths[index], training_params, output_classes[index])

    temp_array = temp_combo[0]
    temp_class = temp_combo[1]
    
    total_tensor_class = total_tensor_class+temp_class
    total_tensor_array = total_tensor_array+temp_array

max_transcript_length = len(max(total_tensor_array, key = len))
total_tensor_array = [entry.ljust(max_transcript_length, "N") for entry in total_tensor_array]
    

total_tensor_array = reads_to_numpy(total_tensor_array)
total_tensor_class = np.asarray(total_tensor_class).astype('float32')


processing: hla_a
/Users/shreyaramesh/Downloads/Shreya_Data/hla_fastm_copy/hla_a2.fastm
1
['/NNNNNNNNNCNNNNNNNNNNTNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNTNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN', '/NNNNNNNNNCNNNNNNNNNNTNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNTNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN', 'UNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNANNNNNNNNNNNNNNCNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNCNNNNNNNNNNNNNNNNNNNNN', 'UNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNANNNNNNNNNNNNNNCNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNCNNNNNNNNNNNNNNNNNNNNN', 'sNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN', 'sNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN', 'eNNNNNNNTNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNCNNNNNNNNNNNNNNNNNNNNNNNNNANNNN

/Users/shreyaramesh/Synkrino/ShreyaNNtraining_Final.py:93: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataframe = pd.read_csv(file, header = None, delimiter = 'bp,')


[[[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 ...

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3.

In [ ]:
print(total_tensor_array)
print(total_tensor_class.shape)

In [ ]:
np.save('data/Sample/TensorArrays/tensor_arrays_5way.npy', total_tensor_array)
np.save('data/Sample/TensorArrays/tensor_class_5way.npy', total_tensor_class)

In [15]:
X = np.load('data/Sample/TensorArrays/tensor_arrays_5way.npy')
y = np.load('data/Sample/TensorArrays/tensor_class_5way.npy')
X.shape
print(X)

[[[1 0 0 0]
  [0 0 0 1]
  [0 0 1 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 1]
  [0 0 1 0]
  [0 0 1 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 1 0]
  [0 0 1 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 ...

 [[1 0 0 0]
  [0 0 1 0]
  [0 0 1 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 1 0]
  [0 0 1 0]
  [1 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 1 0]
  [1 0 0 0]
  [0 0 1 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape, y_train.shape)
print(X_train)

# determine the number of input features
n_features = X_train.shape[1]

#encoder = LabelEncoder()
#encoder.fit(y_test)
#encoder.fit(y_train)
#encoded_ytrain=encoder.transform(y_train)
#encoded_ytest=encoder.transform(y_test)

(7892, 136, 4) (7892,)
[[[0 0 1 0]
  [1 0 0 0]
  [0 0 1 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 1 0 0]
  [1 0 0 0]
  [1 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[1 0 0 0]
  [0 0 1 0]
  [1 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 ...

 [[0 1 0 0]
  [1 0 0 0]
  [0 0 1 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 1 0]
  [0 0 1 0]
  [1 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 1]
  [0 0 1 0]
  [1 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]


## Working NN

In [22]:
model = tf.keras.Sequential()#[tf.keras.layers.Flatten(input_shape=(n_features, 4))]) 

model.add(tf.keras.layers.Conv1D(filters = 320, kernel_size = 16, 
                                 strides=3, padding='valid', activation='relu', input_shape=(n_features,4)))

model.add(layers.MaxPool1D(pool_size=3, strides=None))

model.add(Dropout(0.5))

model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32), merge_mode = 'concat'))# merge_mode='concat', weights=None, backward_layer=None))

model.add(Dropout(0.5))

model.add(Dense(4, activation='softmax', kernel_initializer='he_normal'))

model.add(Dropout(0.5))

model.add(Dense(1, activation='softmax', kernel_initializer='he_normal'))
#model.add(Dense(1, activation='softmax', kernel_initializer='he_normal'))

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

model.compile(optimizer=optimizer, loss='SparseCategoricalCrossentropy', metrics=['accuracy'])
#model.compile(optimizer=optimizer, loss='BinaryCrossentropy', metrics=['accuracy'])

model.summary()

log_dir = "/Users/shreyaramesh/Downloads/Shreya_Data/hla_fastm_copy/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


model.fit(X_train, y_train, validation_data=(X_test, encoded_ytest), batch_size=8, epochs=50, verbose = 1, shuffle = True, callbacks = [tensorboard_callback])

#y_predict = model.fit(X_train, y_train).predict(X_test)
loss, acc = model.evaluate(X_test, encoded_ytest, verbose=1)
print('Test Accuracy: %.3f' % acc)

NotImplementedError: Cannot convert a symbolic Tensor (bidirectional_4/forward_lstm_4/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [11]:
tensorboard --logdir '/Users/shreyaramesh/Downloads/Shreya_Data/hla_fastm_copy/logs/fit/'

## Confusion Matrix and Classification Report

In [ ]:
#Link: https://www.machinecurve.com/index.php/2020/02/21/how-to-predict-new-samples-with-your-keras-model/
predictions = model.predict(X_test)

#Getting the max probabilities for the predictions
y_pred_classes = np.argmax(predictions, axis = 1)
#print(y_pred_classes)

result = tf.math.confusion_matrix(y_test,y_pred_classes)

 
# Printing the result
print('Confusion_matrix: ',result )
 

#print('Confusion Matrix\n')
#print(confusion)

classification_report(y_test, y_pred_classes)
print('\nClassification Report\n')
print(classification_report(y_test, y_pred_classes, target_names=['non-hla', 'hla_a', 'hla_dp', 'hla_dq', 'hla_dr']))


## Grid Search

In [ ]:
## NN Parameters for tuning
#alphas = np.array([0.2,0.1,0.01,0.05,0.001])
#learningrateinit = np.array([0.2,0.1,0.01,0.001])
#hiddenlayers = np.array([50,100,200,300])
#maxiter = np.array([100,200,300])
#solvers = ('adam','lbfgs') #('lbfgs', 'sgd', 'adam')
#activations = ('relu') #,'logistic','tanh','identity')  # ('identity', 'logistic', 'tanh', 'relu')
#param = dict( activation=activations, solver=solvers, alpha = alphas, hidden_layer_sizes = hiddenlayers, max_iter = maxiter, learning_rate_init = learningrateinit)


#learningrateinit = np.array([0.1,0.01])
#hiddenlayers = np.array([50,100,200])
#param = dict(hidden_layer_sizes = hiddenlayers, learning_rate_init = learningrateinit)

#grid = GridSearchCV(estimator=model, param_grid=param,
#                    cv=5, verbose=10)
#grid = GridSearchCV(model,param, cv=5)
#grid.fit(X, y)


## Original NN Code

In [ ]:
'''
num_classes = len(output_classes.keys())
for train_class, idx in output_classes.items():
    output_arr = np.zeros(num_classes)
    output_arr[idx] = 1
    train_sampler = FastmSampler(fastm_file_path=output_paths[train_class],
                        n_prob_upper_limit=training_params['n_prob_upper_limit'],
                        mutation_prob_upper_limit=training_params['mutation_prob_upper_limit'],
                        delete_prob_upper_limit=training_params['delete_prob_upper_limit'],
                        insert_prob_upper_limit=training_params['insert_prob_upper_limit'],
                        min_read_length=training_params['min_read_length'],
                        tensor_length=mp['max_fragment_length'],
                        b_randomize_location=training_params['b_randomize_location'],
                        b_randomize_direction=training_params['b_randomize_direction'],
                        training_output=output_arr
                       )
    print(train_class,idx,output_arr)
    fastm_samplers.append(train_sampler)
'''

In [ ]:
'''
def shuffle_x_y_together(a, b):
    """
    Shuffles 2 numpy arrays together at the 1st dimension. Keeps them together, such that (Xi,Yi) => (Xj,Xj)
    :param a: Numpy Array
    :param b: Numpy Array
    :return: Shuffled A, Shuffled B
    """
    a_s = a.shape[0]
    b_s = b.shape[0]
    assert a_s == b_s, "Arrays must be of same size"
    i = np.argsort(np.random.permutation(a_s))
    return a[i, :, :], b[i, :]

shuffle_x_y_together(total_tensor_array, total_tensor_class)


def build_train_sets(data_samplers,
                    sample_size=1000,
                     test_size=.2):
    """
    Builds creates a train/test tensor for a round of training. 
    :param data_samplers: list sfseventools.FastmSampler
    :param sample_size: Samples per tensor, default 1000
    :param test_size: Test size as percentage of sample size. Default .2
    :return: x_train, y_train, x_test, y_test
    """
    length = len(data_samplers)
    test_length = int(length * test_size)

    x_train = np.concatenate([data_samplers[i].build_tensor(sample_size) 
                                                for i in range(length)])
    y_train = np.concatenate([data_samplers[i].build_output_tensor(sample_size) for i in range(length)])
    x_test = np.concatenate([data_samplers[i].build_tensor(sample_size) for i in range(test_length)])
    y_test = np.concatenate([data_samplers[i].build_output_tensor(sample_size) for i in range(test_length)])


    return x_train, y_train, x_test, y_test

build_train_sets(total_tensor_array, 1000, 0.2)

'''

In [ ]:
def print_log(file_path,epoch):
    f = open(file_path,'a')
    f.write('Trained epoch: '+ epoch +'\n') # python will convert \n to os.linesep
    f.close()

# Log Training
def train_log(file_path, epoch, accuracy):
    f = open(file_path,'a')
    f.write('Epoch '+str(epoch)+ ' loss/accuracy:'+ str(accuracy)+'\n')
    print('Epoch '+str(epoch)+ ' loss/accuracy:'+ str(accuracy)+'\n')# python will convert \n to os.linesep
    f.close()

In [ ]:
def train_network(model,
                  data_sets,
                  output_file, 
                  log_filepath,
                  training_params,
                  history,
                  save_weights = True
                  ):

    test_loss = []
    test_acc = []
    
    for i in range(training_params['nb_epochs']):
        X_train,y_train,X_test,y_test = build_train_sets(data_samplers=fastm_samplers,
                  sample_size = training_params['num_samples'])
        print('epoch: ' + str(i))
        

       
        model.fit(X_train, y_train, batch_size=training_params['batch_size'],
                  epochs=1,
                  verbose=0, shuffle=True, callbacks=[history])    # Manually save model
        loss_and_metrics = model.evaluate(X_test, y_test, batch_size=training_params['batch_size'])
        test_loss.append(loss_and_metrics[0])
        test_acc.append(loss_and_metrics[1])
        train_log(log_filepath,str(i),loss_and_metrics)

        if save_weights and i%10 ==0:
            print("Saving Model")
            model.save_weights(output_file,overwrite=True)


In [ ]:
train_network(model=model, 
              data_sets=fastm_samplers,
              output_file = "/root/Notebooks/test/keras_files/hla_weights.h5",
              log_filepath= "/root/Notebooks/test/keras_files/log_file.txt",
              training_params = training_params,
              history=history
             )

In [ ]:
with open("/root/Notebooks/test/keras_files/hla_network_schema.json", "w") as json_file:
    json_file.write(model.to_json())



In [ ]:
model.save_weights("/root/Notebooks/test/keras_files/hla_weights_200epoch.h5",overwrite=True)

In [ ]:
res_x = x[10001]
res_y = y[10001]
tmp_x, tmp_y = shuffle_x_y(x,y)
for idx, arr in enumerate(tmp_x):
    if (arr == res_x).all():
        print (res_y, tmp_y[idx],idx)

In [ ]:
a = np.concatenate([fastm_samplers[i].build_tensor(15) for i in range(len(fastm_samplers))])

In [ ]:
len(fastm_samplers)

In [ ]:
np.random.choice(100,3)

In [ ]:
from random import uniform
int(round(uniform(1,18), 0))

In [ ]:
np.zeros(len(output_classes.keys()))

In [ ]:
for k,v in output_classes.items():
    print(k,v)

In [ ]:
a = np.array([
    [[0, 0, 0, 0, 0],
    [ 27,  0,  0,  0, 0]], 
    
    [[1, 1, 1, 1, 1],
    [ 1,  18, 1 ,  1, 1]],
    
    [[2, 2, 2, 2, 2],
    [ 2,  2,  22,  2, 2]], 
    
    [[3, 3, 3, 3, 3],
    [ 3,  3,  3,  33, 3]],
    
    [[4, 4, 4, 4, 4],
    [ 4,  4,  4,  4, 44]], 
    
    [[5, 5, 5, 5, 5],
    [ 5,  1,  5,  5, 55]],
])
b = np.array([[0,0,0],
              [1,0,0],
              [0,2,0],
              [0,3,0],
              [0,0,4],
              [0,0,5]])

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
s_a, s_b = shuffle_x_y(a,b)

In [ ]:
s_a

In [ ]:
s_b

In [ ]:
a

In [ ]:
t = np.array([[[  0.,   1.,   2.],
                  [  3.,   4.,   5.]],

                 [[  6.,   7.,   8.],
                  [  9.,  10.,  11.]],

                 [[ 12.,  13.,  14.],
                  [ 15.,  16.,  17.]]])

In [ ]:
t[[0,2,1],:,:]

In [ ]:
np.asarray([train_negative.training_output for i in range(15)])